<a href="https://colab.research.google.com/github/tanjascats/BeyondCoding-workshop/blob/main/BeyondCoding_workshop_24062022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beyond Coding Workshop: 
## L7: Privacy from a developer's perspective

### Contents:
1. Anonymisation using crowds
2. Synthetic data generation

In [2]:
!pip install crowds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt

from crowds.kanonymity.ola import anonymize
from crowds.kanonymity.information_loss import dm_star_loss 
from crowds.kanonymity.generalizations import GenRule

In [2]:
# import data
adult = pd.read_csv('adult.csv', sep=' *; *', na_values='?', engine='python')
adult.info()

In [7]:
# one of these two hierarchies I can write with them; and another one they try to write on their own
def generalise_marital_status(value):
  if value in ['Married-AF-spouse','Married-civ-spouse']:
    return 'spouse-present'
  elif value in ['Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent']:
    return 'spouse-not-present'

In [8]:
def generalise_workclass(value):
  if value in ['Private', 'Self-emp-not-inc', 'Self-emp-inc']:
    return 'Non-Government'
  elif value in ['Local-gov', 'Federal-gov', 'State-gov']:
    return 'Government'
  elif value in ['Without-pay', 'Never-worked']:
    return 'Unemployed'

In [9]:
# the below generalisations will be uploaded from the json-s
def generalise_education_level1(value):
  # load dict of the first level
  # dict level_1
  with open('education_level1.json', 'rb') as infile:
    level_1 = json.load(infile)
  if value in level_1:
    return level_1[value]

In [20]:
def generalise_education_level2(value):
  with open('education_level2.json', 'rb') as infile:
    level_2 = json.load(infile)
  if value in level_2:
    return level_2[value]
  else:
    print(value)

In [11]:
def generalise_age_level1(value):
  # this is slow because it needs to open the file every time when transforming a value
  with open('age_level1.json', 'rb') as infile:
    level1 = json.load(infile)
  if str(value) in level1:
    return level1[str(value)]

In [12]:
def generalise_age_level2(value):
  with open('age_level2.json', 'rb') as infile:
    level2 = json.load(infile)
  if str(value) in level2:
    return level2[str(value)]

In [13]:
def generalise_age_level3(value):
  with open('age_level3.json', 'rb') as infile:
    level3 = json.load(infile)
  if str(value) in level3:
    return level3[str(value)]

In [14]:
def generalise_native_country(value):
  with open('native-country_level1.json', 'rb') as infile:
    level1 = json.load(infile)
  if value in level1:
    return level1[value]

idea: compare anonymised data with the one from ARX

In [22]:
generalization_rules = {
    'age': GenRule([generalise_age_level1, generalise_age_level2, generalise_age_level3]), # 4-levels generalization
    'sex': GenRule([]), # 1 level
    'race': GenRule([]), # 1 level
    'marital-status': GenRule([generalise_marital_status]), # 2 levels
    'education': GenRule([generalise_education_level1, generalise_education_level2]), # 3 levels
    'native-country': GenRule([generalise_native_country]), # 2 levels
    'workclass': GenRule([generalise_workclass]) # 2 levels
}

anonymize(adult, generalization_rules=generalization_rules, k=2, max_sup=0.0, info_loss=dm_star_loss)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   sex             30162 non-null  object
 1   age             30162 non-null  int64 
 2   race            30162 non-null  object
 3   marital-status  30162 non-null  object
 4   education       30162 non-null  object
 5   native-country  30162 non-null  object
 6   workclass       30162 non-null  object
 7   occupation      30162 non-null  object
 8   salary-class    30162 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.1+ MB


[         age  workclass  fnlwgt               education  education-num  \
 0      30-39  State-gov   77516           Undergraduate             13   
 1      30-39  State-gov  170525           Undergraduate             10   
 2      30-39  State-gov  143774           Undergraduate             10   
 3      30-39  State-gov  301526           Undergraduate             10   
 4      30-39  State-gov   60227           Undergraduate             13   
 ...      ...        ...     ...                     ...            ...   
 31594  40-49    Private  126850  Professional Education             15   
 31595  50-59  Local-gov  212856                Graduate             14   
 31596  50-59  Local-gov  268213                Graduate             14   
 31597  40-49    Private  191098           Undergraduate             13   
 31598  40-49    Private  304570           Undergraduate             13   
 
            marital-status       occupation    relationship  race   sex  \
 0      spouse-not-pres